In [ ]:
import pandas as pd
import os, gc, sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from sklearn.metrics import roc_curve,auc, precision_recall_curve
from sklearn.model_selection import KFold
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score


sys.path.append("../source/")
from data_loader import *


In [3]:
useID = True

dl = DataLoader()
df_tr = dl.load_csv("test_transaction.csv")
# df_tr_test = dl.load_csv("test_transaction.csv")

if useID:
    dl.transaction = False
    df_id = dl.load_csv("train_identity.csv")

    df = df_tr.merge(df_id, how='left', left_index=True, right_index=True)

    del df_id, df_tr #, df_tr_test

else:
    df = df_tr
    #df_test = df_tr_test
    del df_tr#, df_tr_test


# df_train['isTrain'] = 1
# df_test['isTrain'] = 0

# df = pd.concat([df_train, df_test])
gc.collect()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Columns: 393 entries, TransactionID to V339
dtypes: float32(376), int16(1), int32(2), object(14)
memory usage: 1.0 GB


/home/alex/Research/FindingFraudsters/FindingFraudsters/notebooks/../source/helper_functions.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["TransactionDay"] = np.floor(df["TransactionDT"] / (24*60*60))
/home/alex/Research/FindingFraudsters/FindingFraudsters/notebooks/../source/helper_functions.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["D1n"] = df["TransactionDay"] - df["D1"]
/home/alex/Research/FindingFraudsters/FindingFraudsters/notebooks/../source/helper_functions.py:38: PerformanceWarning: DataFrame is

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Columns: 439 entries, TransactionID to R_emaildomain_FE
dtypes: bool(9), float32(418), float64(2), int16(1), int32(2), int8(6), object(1)
memory usage: 859.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  144233 non-null  int32  
 1   id_01          144233 non-null  float32
 2   id_02          140872 non-null  float32
 3   id_03          66324 non-null   float32
 4   id_04          66324 non-null   float32
 5   id_05          136865 non-null  float32
 6   id_06          136865 non-null  float32
 7   id_07          5155 non-null    float32
 8   id_08          5155 non-null    float32
 9   id_09          74926 non-null   float32
 10  id_10          74926 non-null   float32
 11  id_11          140978 non-null  float32
 12  id_12          

20

In [3]:
with open("drop_columns.txt") as f:
    to_drop = f.readlines()

to_drop = [item.strip() for item in to_drop]
df = df.drop(columns=to_drop)


In [4]:
df["D1n"] = df["TransactionDay"] - df["D1"]
df['uid'] = df["card1"].astype(str)+'_'+df["addr1"].astype(str)+'_'+df["D1n"].astype(str)
# df_train = df_train.drop(columns=["TransactionID"])transactionID = df['TransactionID']
TransactionID = df['TransactionID'].values
df = df.drop(columns=["TransactionID"])

/tmp/ipykernel_12686/1998728245.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["D1n"] = df["TransactionDay"] - df["D1"]
/tmp/ipykernel_12686/1998728245.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['uid'] = df["card1"].astype(str)+'_'+df["addr1"].astype(str)+'_'+df["D1n"].astype(str)


In [5]:
# Sort by TransactionTD
df = df.sort_values('TransactionDT').reset_index(drop=True)

# Create an empty column
df['IsDuplicateInWindow'] = 1

# Get numpy arrays for faster operations
tds = df['TransactionDT'].values
amts = df['TransactionAmt'].values

# Use searchsorted to find window ranges
for i in range(len(df)):
    lower = tds[i] - 500
    upper = tds[i] + 500

    # Find indices where TD is within ±100
    start = np.searchsorted(tds, lower, side='left')
    end = np.searchsorted(tds, upper, side='right')

    # Slice the relevant window and check for other matching TransactionAmt
    window_amts = amts[start:end]
    match_count = np.sum(window_amts == amts[i])

    df.at[i, 'IsDuplicateInWindow'] = match_count


In [6]:
columns_to_encode = []
# columns_to_encode = [v for v in df.columns if v.startswith("V")]
columns_to_encode.append("TransactionAmt")
columns_to_encode.append("TransactionDT")
d_columns = [d for d in df.columns if d.startswith("D")]
columns_to_encode += d_columns
df = encode_AG(df, 'uid', columns_to_encode)

columns_to_encode = ["addr1", "card1", "card2", "card3", "P_emaildomain", "R_emaildomain"]
df = encode_FE(df, columns_to_encode)



In [10]:
model = xgb.Booster()
model.load_model('model.json')

# df = df.drop(columns=["uid", "card1", "addr1", "D1"])
X = df.drop(columns=["uid"])

dval = xgb.DMatrix(X)
y_prob = model.predict(dval)



In [ ]:
output_df = pd.DataFrame(data=zip(TransactionID,y_prob), columns=["TransactionID", "isFraud"])
output_df.to_csv("test_predictions.csv", index=False)